# 제로샷 텍스트 분류

클래스 이름은 이미 분류하려는 텍스트를 잘 설명하고 있을 가능성이 높습니다. 🤗 SetFit을 사용하면 이러한 클래스 이름을 강력한 사전 훈련된 Sentence Transformer 모델과 함께 사용하여 훈련 샘플 없이도 강력한 기준 모델을 얻을 수 있습니다.

이 가이드에서는 제로샷 텍스트 분류를 수행하는 방법을 보여줍니다.

## 테스트 데이터 세트

제로샷 모델의 성능을 테스트하기 위해 [dair-ai/emotion](https://huggingface.co/datasets/dair-ai/emotion) 데이터 세트를 사용합니다.

In [ ]:
from datasets import load_dataset

test_dataset = load_dataset("dair-ai/emotion", "split", split="test")

이 데이터 세트는 데이터 세트 `Features` 내에 클래스 이름을 저장하므로 다음과 같이 클래스를 추출합니다.

In [ ]:
classes = test_dataset.features["label"].names
# => ['sadness', 'joy', 'love', 'anger', 'fear', 'surprise']

그렇지 않으면 클래스 목록을 수동으로 설정할 수 있습니다.

## 합성 데이터 세트

그런 다음 [get_templated_dataset()](https://huggingface.co/docs/setfit/main/en/reference/utility#setfit.get_templated_dataset)를 사용하여 이러한 클래스 이름을 지정하여 더미 데이터 세트를 합성적으로 생성할 수 있습니다.

In [ ]:
from setfit import get_templated_dataset

train_dataset = get_templated_dataset()

In [ ]:
print(train_dataset)
# => Dataset({
#     features: ['text', 'label'],
#     num_rows: 48
# })
print(train_dataset[0])
# {'text': '이 문장은 슬픔입니다', 'label': 0}

## 훈련

이 데이터 세트를 사용하여 일반적인 SetFit 모델을 훈련할 수 있습니다.

In [ ]:
from setfit import SetFitModel, Trainer, TrainingArguments

model = SetFitModel.from_pretrained("BAAI/bge-small-en-v1.5")

args = TrainingArguments(
    batch_size=32,
    num_epochs=1,
)

trainer = Trainer(
    model=model,
    args=args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
)
trainer.train()

```
***** 훈련 실행 중 *****
  예제 수 = 60
  에포크 수 = 1
  총 최적화 단계 = 60
  총 훈련 배치 크기 = 32
{'embedding_loss': 0.2628, 'learning_rate': 3.3333333333333333e-06, 'epoch': 0.02}                                                                                 
{'embedding_loss': 0.0222, 'learning_rate': 3.7037037037037037e-06, 'epoch': 0.83}                                                                                 
{'train_runtime': 15.4717, 'train_samples_per_second': 124.098, 'train_steps_per_second': 3.878, 'epoch': 1.0}                                                     
100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 60/60 [00:09<00:00,  6.35it/s]
```

훈련이 끝나면 모델을 평가할 수 있습니다.

In [ ]:
metrics = trainer.evaluate()
print(metrics)

```
***** 평가 실행 중 *****
{'accuracy': 0.591}
```

그리고 예측을 실행합니다.

In [ ]:
preds = model.predict([
    "나는 그냥 짜증나고 우울해",
    "그녀와 이야기할 수 있다는 것만으로도 엄청나게 운이 좋다고 느껴",
    "너 지금 나를 화나게 하고 있어",
    "나는 확실히 해양 공포증이 있어, 그런 물 근처에 나를 데려가지 마",
    "전혀 예상하지 못했어",
])
print([classes[idx] for idx in preds])

In [ ]:
['sadness', 'joy', 'anger', 'fear', 'surprise']

이러한 예측은 모두 정확해 보입니다!

## 기준선

SetFit의 제로샷 성능이 잘 작동하는지 보여주기 위해 `transformers`의 제로샷 분류 모델과 비교합니다.

In [ ]:
from transformers import pipeline
from datasets import load_dataset
import evaluate

# 테스트 데이터 세트 준비
test_dataset = load_dataset("dair-ai/emotion", "split", split="test")
classes = test_dataset.features["label"].names

# transformers에서 제로샷 분류 파이프라인 설정
# 기본적으로 'facebook/bart-large-mnli' 사용
pipe = pipeline("zero-shot-classification", device=0)
zeroshot_preds = pipe(test_dataset["text"], batch_size=16, candidate_labels=classes)
preds = [classes.index(pred["labels"][0]) for pred in zeroshot_preds]

# 정확도 계산
metric = evaluate.load("accuracy")
transformers_accuracy = metric.compute(predictions=preds, references=test_dataset["label"])
print(transformers_accuracy)

In [ ]:
{'accuracy': 0.3765}

59.1%의 정확도로 0샷 SetFit은 `transformers`에서 권장하는 제로샷 모델보다 훨씬 뛰어난 성능을 보입니다.

## 예측 대기 시간

더 높은 정확도를 얻는 것 외에도 SetFit은 훨씬 빠릅니다. `BAAI/bge-small-en-v1.5`를 사용한 SetFit의 대기 시간과 `facebook/bart-large-mnli`를 사용한 `transformers`의 대기 시간을 계산해 보겠습니다. 두 테스트 모두 GPU에서 수행되었습니다.

In [ ]:
import time

start_t = time.time()
pipe(test_dataset["text"], batch_size=32, candidate_labels=classes)
delta_t = time.time() - start_t
print(f"`transformers`와 `facebook/bart-large-mnli`의 대기 시간: 문장당 {delta_t / len(test_dataset['text']) * 1000:.4f}ms")

```
`transformers`와 `facebook/bart-large-mnli`의 대기 시간: 문장당 31.1765ms
```

In [ ]:
import time

start_t = time.time()
model.predict(test_dataset["text"])
delta_t = time.time() - start_t
print(f"SetFit과 `BAAI/bge-small-en-v1.5`의 대기 시간: 문장당 {delta_t / len(test_dataset['text']) * 1000:.4f}ms")

```
SetFit과 `BAAI/bge-small-en-v1.5`의 대기 시간: 문장당 0.4600ms
```

따라서 `BAAI/bge-small-en-v1.5`를 사용한 SetFit은 `facebook/bart-large-mnli`를 사용한 `transformers`보다 67배 빠르며 정확도도 더 높습니다.

![zero_shot_transformers_vs_setfit](https://github.com/huggingface/setfit/assets/37621491/33f574d9-c51b-4e02-8d98-6e04e18427ef)